# Model of YAP/TAZ transport with nuclear envelope rupture

Here, we partially solve the mechanotransduction model (see mechanotransduction.ipynb for full description). In this case, we assume that the cytoskeletal activation has already been solved for, in which case we only need to solve for alterations in nuclear transport of YAP/TAZ (enhanced opening due to stretch, rupture, etc.).

Previous results are loaded in from simulations of YAP/TAZ mechanotransduction. In particular, several variables are loaded in as parameters varying over space and time:
* Myo_A: activated myosin (associated with stress fiber formation) in the cytosol
* FActin: polymerized actin in the cytosol
* LaminA: dephosphorylated form of Lamin A in the nuclear envelope, a structural componet of the nuclear lamina
* NPC_A: activated (open) nuclear pore complexes in the nuclear envelope

There remain 4 species to solve for. 2 in the cytosol:
* YAPTAZ: dephosphorylated form of YAP/TAZ in the cytosol
* YAPTAZ_phos: phosphorylated form of YAP/TAZ (preferentially remains cytosolic)
And 2 in the nucleus:
* YAPTAZ_nuc: concentration of YAP/TAZ in the nucleus

In [ ]:
import dolfin as d
import sympy as sym
import numpy as np
import pathlib
import logging
import argparse
import matplotlib.pyplot as plt


from smart import config, mesh, model, mesh_tools
from smart.units import unit
from smart.model_assembly import (
    Compartment,
    Parameter,
    Reaction,
    Species,
    sbmodel_from_locals,
)
import sys

from mech_parser_args import add_mechanotransduction_nucOnly_arguments
here = pathlib.Path.cwd()
sys.path.insert(0, (here / ".." / "scripts").as_posix())
import runner as main_run

smart_logger = logging.getLogger("smart")
smart_logger.setLevel(logging.DEBUG)
logger = logging.getLogger("mechanotransduction")
logger.setLevel(logging.INFO)
logger.info("Starting mechanotransduction example")

If running a series of tests, we use `argparse` to read in the test conditions. Otherwise, we load default parameters to the `args` dict.

In [ ]:
sys.path.insert(0, "/root/shared/gitrepos/smart-nanopillars/utils")
parser = argparse.ArgumentParser()
add_mechanotransduction_nucOnly_arguments(parser)
try:
    shell = get_ipython().__class__.__name__
    args = {}
    if shell == 'ZMQInteractiveShell': # then running a jupyter notebook
        args["outdir"] = pathlib.Path("results_poresims")
        args["time_step"] = 0.01
        args["pore_size"] = 0.1
        args["pore_loc"] = 0.0
        args["pore_rate"] = 10.0
        args["transport_rate"] = 10000.0
        args["transport_ratio"] = 10.0
        args["a0_npc"] = 5.0
        args["nuc_compression"] = 2.8
        args["alt_yap_diffusion"] = False
        cur_dir = str(pathlib.Path.cwd() / "..")
        args["full_sims_folder"] = pathlib.Path(f"{cur_dir}/analysis_data/simulation_results_2.8indent")
        mesh_folder = pathlib.Path(f"{cur_dir}/meshes/nanopillars_indent/nanopillars_indent2.8")
        args["mesh_folder"] = mesh_folder
        full_sim_folder_cur = args["full_sims_folder"]
except:
    args = {}
if len(args) == 0: # then most likely running as a script
    args = vars(parser.parse_args())
    full_sim_folder_cur = (args["full_sims_folder"] / 
                           f"nanopillars_indent{args['nuc_compression']}_a0_{args['a0_npc']}")
    if not full_sim_folder_cur.exists():
        full_sim_folder_cur = args["full_sims_folder"]

# hEdge = 0.5
# hNP = hEdge * 0.1
nanopillars = [0.5, 3.0, 3.5]
timer = d.Timer("mechanotransduction-example")

Now we define units for use in the model.

In [ ]:
# Aliases - base units
uM = unit.uM
um = unit.um
molecule = unit.molecule
sec = unit.sec
dimensionless = unit.dimensionless
# Aliases - units used in model
D_unit = um**2 / sec
flux_unit = uM * um / sec
vol_unit = uM
surf_unit = molecule / um**2
kPa = unit.kilopascal

# Model generation

For each step of model generation, refer to SMART Example 3 or API documentation for further details.

In this file, two different trasport rates and 3 different transport ratios (upsilon in main text) are tested for a single pore radius. Nuclear rupture occurs directly above the central nanopillar.

In [ ]:
Cyto = Compartment("Cyto", 3, um, 1)
Nuc = Compartment("Nuc", 3, um, 2)
NM = Compartment("NM", 2, um, 12)

if args["alt_yap_diffusion"]:
    cyto_yap_diff = 80.0
    nuc_yap_diff = 4.0
else:
    cyto_yap_diff = 19.0
    nuc_yap_diff = 19.0


YAPTAZ = Species(
    "YAPTAZ", 0.7, vol_unit, cyto_yap_diff, D_unit, "Cyto"
)  # non-phosphorylated in the cytosol
YAPTAZ_phos = Species(
    "YAPTAZ_phos", 0.2, vol_unit, cyto_yap_diff, D_unit, "Cyto"
)  # phosphorylated in the cytosol
YAPTAZ_nuc = Species("YAPTAZ_nuc", 0.7, vol_unit, nuc_yap_diff, D_unit, "Nuc")

parent_mesh = mesh.ParentMesh(
    mesh_filename=str(pathlib.Path(args["mesh_folder"]) / "spreadCell_mesh.h5"),
    mesh_filetype="hdf5",
    name="parent_mesh",
    curvature=pathlib.Path(args["mesh_folder"]) / "curvatures.xdmf",
    extra_keys=["subdomain0_2"]
)

transport_ratio_vec = [1, 2, 5]
parent_dir = args["outdir"]
args["pore_loc"] = 0.0
args["pore_rate"] = 1.0
for i in range(len(transport_ratio_vec)):
    args["transport_ratio"] = transport_ratio_vec[i]
    args["outdir"] = (parent_dir / f"nanopillars_indent{args['nuc_compression']}_pore_size{args['pore_size']}"
                                   f"_loc0.0_rate1_transport{args['transport_rate']}_ratio{args['transport_ratio']}")
    # Module C: nucleo-cytoplasmic transport
    # c1: YAP/TAZ dephos. and phos. in the cytosol
    k_CN = Parameter("k_CN", 0.56, 1 / sec)
    k_CY = Parameter("k_CY", 0.00076, 1 / (sec * uM**2))
    k_NC = Parameter("k_NC", 0.14, 1 / sec)
    FActin = Parameter.from_xdmf("FActin", full_sim_folder_cur / "FActin.xdmf", uM, "Cyto")
    Myo_A = Parameter.from_xdmf("Myo_A", full_sim_folder_cur / "Myo_A.xdmf", uM, "Cyto")
    if args["a0_npc"] > 0 and args["nuc_compression"] > 0:
        aNPC = Parameter.from_xdmf("aNPC", full_sim_folder_cur / "aNPC.xdmf", dimensionless, "NM")
    c1 = Reaction(
        "c1",
        ["YAPTAZ_phos"],
        ["YAPTAZ"],
        param_map={"k_CN": "k_CN", "k_CY": "k_CY", "k_NC": "k_NC", 
                "FActin": "FActin", "Myo_A": "Myo_A",},
        species_map={
            "YAPTAZ": "YAPTAZ",
            "YAPTAZ_phos": "YAPTAZ_phos",
        },
        eqn_f_str="YAPTAZ_phos*(k_CN + k_CY*FActin*Myo_A) - k_NC*YAPTAZ",
    )

    NPC_A = Parameter.from_xdmf("NPC_A", full_sim_folder_cur / "NPC_A.xdmf", surf_unit, "NM")

    # c5: nuclear translocation of YAP/TAZ
    k_insolo = Parameter("k_insolo", 1.0, surf_unit / (sec * uM))
    k_in2 = Parameter("k_in2", 10.0, 1 / (sec * uM))
    k_out = Parameter("k_out", 1.0, surf_unit / (sec * uM))
    if args["a0_npc"] > 0 and args["nuc_compression"] > 0:
        a0_NPC = Parameter("a0_NPC", args["a0_npc"], dimensionless)
        c5 = Reaction(
            "c5",
            ["YAPTAZ"],
            ["YAPTAZ_nuc"],
            param_map={"k_insolo": "k_insolo", "k_in2": "k_in2", "k_out": "k_out", 
                    "a0_NPC": "a0_NPC", "NPC_A": "NPC_A", "aNPC": "aNPC"},
            species_map={"YAPTAZ": "YAPTAZ", "YAPTAZ_nuc": "YAPTAZ_nuc"},
            explicit_restriction_to_domain="NM",
            eqn_f_str="YAPTAZ*exp((aNPC-1)/a0_NPC)*(k_insolo + k_in2*NPC_A) - k_out*YAPTAZ_nuc",
        )
    else:
        c5 = Reaction(
            "c5",
            ["YAPTAZ"],
            ["YAPTAZ_nuc"],
            param_map={"k_insolo": "k_insolo", "k_in2": "k_in2", "k_out": "k_out", "NPC_A": "NPC_A"},
            species_map={"YAPTAZ": "YAPTAZ", "YAPTAZ_nuc": "YAPTAZ_nuc"},
            explicit_restriction_to_domain="NM",
            eqn_f_str="YAPTAZ*(k_insolo + k_in2*NPC_A) - k_out*YAPTAZ_nuc",
        )

    # c7: nuclear rupture events
    xPore = args["pore_loc"]
    xMax = nanopillars[2]*np.round(100./nanopillars[2])
    np_loc = np.linspace(0.0, xMax, int(np.round(100./nanopillars[2]))+1)
    assert np.min(np.abs(xPore-np_loc)) < nanopillars[0], "Pore location should be over nanopillar"
    yPore = 0.0
    zPore = nanopillars[1] + 0.2
    pore = Parameter.from_expression("pore", 
                                        f"(1-exp(-t/{args['pore_rate']}))*"
                                        f"exp(-(pow(x-{xPore},2)+pow(y-{yPore},2)+pow(z-{zPore},2))"
                                        f"/(pow({args['pore_size']},2)))", 
                                        dimensionless)

    # c7 and c8: YAP leak due to lysis
    k_inPore = Parameter("k_inPore", args["transport_ratio"]*args["transport_rate"], surf_unit / (sec * uM))
    k_outPore = Parameter("k_outPore", args["transport_rate"], surf_unit / (sec * uM))
    c7 = Reaction(
        "c7",
        ["YAPTAZ"],
        ["YAPTAZ_nuc"],
        param_map={"k_inPore": "k_inPore", "k_outPore": "k_outPore", "pore": "pore"},
        species_map={"YAPTAZ": "YAPTAZ", "YAPTAZ_nuc": "YAPTAZ_nuc"},
        explicit_restriction_to_domain="NM",
        eqn_f_str="YAPTAZ*pore*k_inPore - k_outPore*pore*YAPTAZ_nuc",
    )

    # set config for current run
    configCur = config.Config()
    configCur.flags.update(
        {
            "allow_unused_components": True,
        }
    )
    configCur.solver.update(
        {
            "final_t": 100000.0,
            "initial_dt": args["time_step"],
            "time_precision": 6,
            "use_snes": True,
            "attempt_timestep_restart_on_divergence": True,
            "reset_timestep_for_negative_solution": True,
        }
    )

    pc, sc, cc, rc = sbmodel_from_locals(locals().values())

    model_cur = model.Model(pc, sc, cc, rc, configCur, parent_mesh)
    model_cur.initialize(initialize_solver=True)
    # Write initial condition(s) to file
    results = dict()
    result_folder = args["outdir"]
    result_folder.mkdir(exist_ok=True, parents=True)

    if model_cur.mpi_comm_world.rank == 0:
        import json
        # Dump config to results folder
        (result_folder / "config.json").write_text(
            json.dumps(
                {
                    "solver": configCur.solver.__dict__,
                    "flags": configCur.flags.__dict__,
                    "reaction_database": configCur.reaction_database,
                    "mesh_file": str(args["mesh_folder"]),
                    "outdir": str(args["outdir"]),
                    "time_step": args["time_step"],
                }
            )
        )

    for species_name, species in model_cur.sc.items:
        results[species_name] = d.XDMFFile(
            model_cur.mpi_comm_world, str(result_folder / f"{species_name}.xdmf")
        )
        results[species_name].parameters["flush_output"] = True
        results[species_name].write(model_cur.sc[species_name].u["u"], model_cur.t)
    # model_cur.to_pickle(result_folder / "model_cur.pkl")

    # Set loglevel to warning in order not to pollute notebook output
    smart_logger.setLevel(logging.WARNING)
    YAPTAZ_nuc_vec = np.array([model_cur.sc["YAPTAZ_nuc"].initial_condition])

    # Solve
    displayed = False
    while True:
        logger.info(f"Solve for time step {model_cur.t}")
        model_cur.monolithic_solve()
        model_cur.adjust_dt()

        nucMesh = model_cur.cc["Nuc"].dolfin_mesh
        dx = d.Measure("dx", domain=nucMesh)
        int_val = d.assemble_mixed(model_cur.sc["YAPTAZ_nuc"].u["u"] * dx)
        volume = d.assemble_mixed(1.0 * dx)
        current_YAPTAZ_nuc = np.array([int_val / volume])
        YAPTAZ_nuc_vec = np.concatenate((YAPTAZ_nuc_vec, current_YAPTAZ_nuc))

        # Save results for post processing
        for species_name, species in model_cur.sc.items:
            results[species_name].write(model_cur.sc[species_name].u["u"], model_cur.t)

        if model_cur.mpi_comm_world.rank == 0: 
            np.savetxt(result_folder / "YAPTAZ_nuc.txt", YAPTAZ_nuc_vec.astype(np.float32))
            np.savetxt(result_folder / "tvec.txt", np.array(model_cur.tvec).astype(np.float32))
        # End if we've passed the final time
        if model_cur.t >= model_cur.final_t:
            break

    logger.info("Done with solve loop")
    timer.stop()
    timings = d.timings(
        d.TimingClear.keep,
        [d.TimingType.wall, d.TimingType.user, d.TimingType.system],
    ).str(True)

    if model_cur.mpi_comm_world.size > 1:
        d.MPI.comm_world.Barrier()

    if model_cur.mpi_comm_world.rank == 0:
        print(timings)
        (result_folder / "timings.txt").write_text(timings)
        plt.plot(model_cur.tvec, YAPTAZ_nuc_vec)
        plt.xlabel("Time (s)")
        plt.ylabel("YAPTAZ_nuc (μM)")
        plt.savefig(result_folder / "YAPTAZ_nuc.png")